In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
myLin = nn.Linear(3,12)

In [10]:
X = torch.rand(1,5,3)
y = torch.rand(1,5,1)

pred = myLin(X)
print(pred[-1])
pred = pred.view(1,5,4,3)
pred.shape

tensor([[-0.2853, -0.1057,  0.4158, -0.9548,  0.0891, -0.4504, -0.1180, -0.5443,
         -0.1376, -0.1873, -0.2776, -0.6254],
        [-0.4549,  0.1568,  0.2583, -1.0162, -0.0182, -0.4958,  0.3076, -0.0196,
          0.0388, -0.4013, -0.2181, -0.3737],
        [-0.3272, -0.1195,  0.2708, -0.8679,  0.0035, -0.4211,  0.0950, -0.2810,
         -0.0226, -0.2548, -0.3052, -0.4274],
        [-0.3951, -0.2980, -0.0724, -0.6055, -0.2240, -0.4621,  0.3429,  0.0369,
          0.2174, -0.3572, -0.4410, -0.0608],
        [-0.1805, -0.4228,  0.1352, -0.5821, -0.0084, -0.0640,  0.3449,  0.0109,
          0.0391, -0.1431, -0.3861, -0.1446]], grad_fn=<SelectBackward0>)


torch.Size([1, 5, 4, 3])

In [30]:
a = torch.randint(5, (1,2,3,2))
print(a, '\n')
b = torch.randint(5, (1,2,3,2))
b = b.transpose(-2,-1)
print(b)

a @ b

tensor([[[[3, 4],
          [1, 3],
          [4, 3]],

         [[3, 0],
          [4, 2],
          [0, 1]]]]) 

tensor([[[[1, 3, 4],
          [0, 2, 3]],

         [[4, 3, 0],
          [1, 3, 0]]]])


tensor([[[[ 3, 17, 24],
          [ 1,  9, 13],
          [ 4, 18, 25]],

         [[12,  9,  0],
          [18, 18,  0],
          [ 1,  3,  0]]]])